In [1]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_qegz213
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_qegz213
  Resolved https://github.com/huggingface/transformers to commit 2890116ab761256c8d7e806c6cbf8f7e841b2abc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8662221 sha256=5acd25f0e683b362b08db15d39e35643074582bf3e29147515ed2fb69eb46a4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-q2tpz43s/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [3]:
!pip install datasets evaluate wandb torchmetrics torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast
from transformers import AutoModelForUniversalSegmentation, Mask2FormerImageProcessor
import torchvision.transforms as tvt
from torchvision.transforms.functional import crop
from torchmetrics.classification import Accuracy, MulticlassAccuracy, MulticlassJaccardIndex, JaccardIndex
import albumentations as A
from tqdm.auto import tqdm
from huggingface_hub import notebook_login
from datasets import load_dataset
import wandb
import evaluate
import statistics
from copy import deepcopy
from huggingface_hub import hf_hub_download
import torchinfo

import numpy as np
import pandas as pd
import random
import requests
import json
from pathlib import Path
import os
from typing import List, Dict, Tuple
import warnings
from PIL import Image as PILImage
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

In [5]:
# import os
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [6]:
#!huggingface-cli login
from huggingface_hub import login
from google.colab import userdata
import wandb

login(token=userdata.get('HF_TOKEN'))
wandb.login(key=userdata.get('WANDB_API_KEY'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
def get_labels():
    data_directory = Path(".")
    json_file_path = data_directory / "labels_info.json"
    if json_file_path.is_file():
        print(f"[INFO]: Found {json_file_path}.Skipping Download...")
    else:
        print("[INFO]: Downloading labels_info.json from hub")
        json_file_path = hf_hub_download(
            repo_id="BhavanaMalla/railsem19-semantic-expanded",
            filename="labels_info.json",
            repo_type="dataset",
            local_dir=data_directory
        )
    with open(json_file_path, "r") as f:
        labels_info = json.load(f)
    id2label = labels_info["id2label"]
    label2id = labels_info["label2id"]
    labels = labels_info["labels"]
    color_palette = labels_info["color_palette"]

    # add the background label
    id2label["19"] = "background"
    label2id["background"] = 19
    labels.append("background")
    color_palette.append([0, 0, 0])

    # correcting the labels
    id2label = {int(key): value.replace('-', '_') \
                for key, value in id2label.items()}
    label2id = {key.replace('-', '_'): value for key, value in label2id.items()}
    labels = [label.replace('-', '_') for label in labels]
    return {"id2label": id2label, "label2id": label2id, "labels": labels,
            "color_palette": color_palette}

In [8]:
def load_railsem_dataset():
    print(f"[INFO]: Extracting Railsem19 dataset from hub on")
    railsem_ds = load_dataset("BhavanaMalla/railsem19-semantic-expanded")
    return railsem_ds

In [9]:
def get_modified_labels():
    modified_id2label = {
        0: "road", 1: "sidewalk", 2: "construction_fence",
        3: "rail_raised_rail_embedded", 4: "pole_traffic_light_traffic_sign",
        5: "sky", 6: "human", 7: "tram_track_rail_track", 8: "car_truck",
        9: "on_rails", 10: "vegetation", 11: "trackbed",
        12: "background_terrain"
    }
    modified_label2id = {
        label: id for id, label in modified_id2label.items()
    }
    modified_labels = [label for label in modified_id2label.values()]
    return modified_id2label, modified_label2id, modified_labels

In [10]:
class RemapBackground():
    """ Remap background to label 19 """
    def __call__(self, mask):
        return torch.where(mask > 18, 19, mask)


class RemapLabels():
    def __init__(self):
        self.class_mapping = get_labels()["id2label"]
        self.class_coding = {v: k for k, v in self.class_mapping.items()}

        # Remapping of original 20 labels to 13 labels
        self.modified_labels = {
            0: ["road"], 1: ["sidewalk"],
            2: ["construction", "fence"],
            3: ["rail_raised", "rail_embedded"],
            4: ["pole", "traffic_light", "traffic_sign"],
            5: ["sky"], 6: ["human"],
            7: ["tram_track", "rail_track"],
            8: ["car", "truck"], 9: ["on_rails"],
            10: ["vegetation"], 11: ["trackbed"],
            12: ["background", "terrain"]
        }
        self.modified_ids = {}
        for k, v in self.modified_labels.items():
            self.modified_ids[k] = [self.class_coding[label] for label in v]

    def __call__(self, mask):
        final_label = np.zeros_like(mask)
        for key, val in self.modified_ids.items():
            specific_label = np.zeros_like(mask)
            specific_label = np.where(np.isin(mask, np.array(val)), 1, 0)
            specific_label *= key
            final_label = np.add(final_label, specific_label)
        return torch.from_numpy(final_label)



In [11]:
class PatchQueue(Dataset):
    '''
    Extract random patches from an image -> put them in queue
    -> feed them to the dataloader
    ref : https://github.com/fepegar/torchio/blob/main/src/torchio/data/queue.py
    '''
    def __init__(self, dataset, max_length,  # Queue Length
                 samples_per_image,  # Patches per image
                 queue_workers=4,  # Num Workers
                 patch_size=[512, 512],  # Patch Size
                 shuffle=False):
        self.dataset = dataset
        self.max_length = max_length
        self.queue_workers = queue_workers
        self.samples_per_image = samples_per_image
        self.shuffle_queue = shuffle
        self.patch_size = patch_size
        self._images_iterable = None
        self.patch_list = list()
        self._num_sampled_images = 0
        self.steps_per_epoch = len(self.dataset) * self.samples_per_image
        self.resize_transform = tvt.Resize(patch_size[::-1])

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, item):
        if not self.patch_list:
            print("[WARN]: Patch List is empty...")
            self._fill()
            self.patch_list.reverse()
        sample_patch = self.patch_list.pop()
        return sample_patch

    @staticmethod
    def _get_first_item(batch):
        return batch[0]

    def initialize_images_iterable(self):
        self._images_iterable = self._get_images_iterable()

    @property
    def images_iterable(self):
        if self._images_iterable is None:
            self.initialize_images_iterable()
        return self.images_iterable

    def _get_images_iterable(self):
        if torch.distributed.is_available() and torch.distributed.is_initialized():
            sampler = torch.utils.data.DistributedSampler(
                self.dataset,
                num_replicas=torch.distributed.get_world_size(),
                rank=torch.distributed.get_rank(), shuffle=self.shuffle_queue
            )
        else:
            sampler = None
        loader = DataLoader(
            self.dataset,  # num_workers=self.queue_workers, #in multigpu, >0 not wrkng
            batch_size=1, collate_fn=self._get_first_item, pin_memory=True,
            shuffle=False, sampler=sampler
        )
        self._num_sampled_images = 0
        return iter(loader)

    def _get_next_image(self):
        try:
            image = next(self._images_iterable)
        except Exception as e:
            print(f"[WARN]:Exception while getting image for patching: {e}")
            self.initialize_images_iterable()
            image = next(self._images_iterable)
        return image

    def extract_patches(self, image_pair, samples_per_image, patch_size):
        image_patches = []
        img_height = image_pair['image'].shape[-2]
        img_width = image_pair['image'].shape[-1]
        # Check if image size is smaller than patch size
        if img_height < patch_size[0] or img_width < patch_size[1]:
            # Resize the image to match the patch size
            image_pair["image"] = self.resize_transform(image_pair["image"])
            image_pair["mask"] = self.resize_transform(image_pair["mask"])
            img_height = image_pair["image"].shape[-2]
            img_width = image_pair["image"].shape[-1]
        for _ in range(samples_per_image):
            left = torch.randint(
                low=0, high=img_width-patch_size[0], size=[1,]
            ).item()
            top = torch.randint(
                low=0, high=img_height-patch_size[1], size=[1,]
            ).item()
            cropped_image = crop(
                img=image_pair["image"], top=top, left=left,
                height=patch_size[1], width=patch_size[0]
            )
            cropped_labels = crop(
                img=image_pair["mask"], top=top, left=left,
                height=patch_size[1], width=patch_size[0]
            )
            image_patches.append({"image": cropped_image,
                                  "mask": cropped_labels})
        return image_patches

    def _fill(self):
        while True:
            image_pair = self._get_next_image()
            samples_per_image = self.samples_per_image
            patch_size = self.patch_size
            patches = self.extract_patches(
                image_pair, samples_per_image, patch_size
            )
            self.patch_list.extend(patches)
            self._num_sampled_images += 1
            islistfull = len(self.patch_list) >= self.max_length
            if islistfull:
                break

In [12]:
def get_transforms():
    # Transforms
    image_transforms = tvt.Compose([tvt.PILToTensor(),])
    mask_transforms = tvt.Compose([tvt.PILToTensor(),
                                  RemapBackground(),
                                  RemapLabels(),])
    return image_transforms, mask_transforms

In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        semantic_mask = self.dataset[idx]["semantic_mask_label"]
        if self.transforms:
            image_transforms, mask_transforms = self.transforms
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        else:
            image_transforms, mask_transforms = get_transforms()
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        return {
            "image": transformed_image, "mask": transformed_mask
        }

In [29]:
def _collate_fn(batch):
    # images = torch.stack([sample["image"] for sample in batch])
    # segmentation_maps = torch.cat([sample["mask"] for sample in batch])
    images = tuple([sample["image"] for sample in batch])
    segmentation_maps = tuple([sample["mask"].squeeze() for sample in batch])
    # batch = processor(images, segmentation_maps=segmentation_maps, task_inputs=["semantic"] * len(images), return_tensors="pt")
    batch = processor(images, segmentation_maps=segmentation_maps, return_tensors="pt")
    batch["original_images"] = images
    batch["original_segmentation_maps"] = segmentation_maps
    return batch

def _prepare_dataloader(dataset: torch.utils.data.Dataset):
    sampler = None
    is_train=True
    dataloader = DataLoader(dataset, batch_size=4, pin_memory=True, shuffle=(is_train and sampler is None), sampler=sampler, collate_fn=_collate_fn)
    return dataloader

In [25]:
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation,  AutoConfig
import torch.nn.functional as F
from torch import nn
import gc
from torch.cuda.amp import GradScaler, autocast


# id2label
id2label, label2id, labels = get_modified_labels()
# model and processor
model_ckpt = "facebook/mask2former-swin-tiny-cityscapes-semantic"
model_config = AutoConfig.from_pretrained(model_ckpt, id2label=id2label, label2id=label2id, num_labels=len(id2label))
model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)

processor = AutoImageProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False, do_resize=False) #384, 384 without do_resize=False
#default - "do_normalize": true, "do_rescale": true, "do_resize": true, "ignore_index": 255, "reduce_labels": false,
processor.num_labels = len(id2label)

#dataset
mydataset = load_railsem_dataset()
total_size = len(mydataset["data"])
print(f"[INFO]: Total images: {total_size}")
train_size = 8160
train_percentage = train_size / total_size
# Use train_test_split with specified percentages
splits = mydataset["data"].train_test_split(train_size=train_percentage, shuffle=True)
train_split = splits["train"]
val_split = splits["test"].select(indices=(range(320))) #320
test_split = splits["test"].select(indices=(range(20)))
print(f"[INFO]: Total Training images: {len(train_split)}")
print(f"[INFO]: Total Validation images: {len(val_split)}")
print(f"[INFO]: Total Test images: {len(test_split)}")

# transforms
transforms = get_transforms()
customdataset = CustomDataset(train_split.select(range(0, 6)), transforms) #1080, 1920
# customdataset = CustomDataset(train_split, transforms)

patch_size = [1024, 1024] #[512, 512] #[1024, 1024]
train_queue = PatchQueue(customdataset, max_length=10, samples_per_image=2, patch_size=patch_size) #1024, 1024
dataloader = _prepare_dataloader(train_queue) #train_queue #customdataset

device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.empty_cache()
    gc.collect()

optimizer = AdamW(params=model.parameters(), lr=1e-4)
scaler = GradScaler()
epochs = 2

model.to(device)
model.train()
for i in range(epochs):
    print(f"------------Epoch: {i}-----------------")
    running_train_loss = 0.0
    for step, batch in enumerate(dataloader):
        print(batch.keys())
        print(f'pixel_values: {batch["pixel_values"].shape}')
        batch_dict = {
                "pixel_values": batch["pixel_values"].to(device),
                "mask_labels": [labels.to(device)
                                for labels in batch["mask_labels"]],
                "class_labels": [labels.to(device)
                                    for labels in batch["class_labels"]],
                "pixel_mask": batch["pixel_mask"].to(device),}
        # with torch.autocast(device_type='cpu', dtype=torch.bfloat16):
        with autocast():
            print(f'mask_labels: {batch["mask_labels"][0].shape}, {batch["mask_labels"][1].shape}') # mask_labels: torch.Size([8, 384, 384]), torch.Size([11, 384, 384])
            print(f'class_labels: {batch["class_labels"][0]}, {batch["class_labels"][1]}') # class_labels: tensor([ 2,  3,  4,  5,  7,  9, 10, 11]), tensor([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 11, 12])
            outputs = model(**batch_dict)
            train_loss = outputs.loss
        # print(outputs.keys())  # dict_keys(['loss', 'class_queries_logits', 'masks_queries_logits', 'encoder_last_hidden_state', 'pixel_decoder_last_hidden_state', 'transformer_decoder_last_hidden_state'])
        print(f"Step: {step}, Train Loss: {train_loss.item()}")
        batch_size = batch["pixel_values"].size(0)
        running_train_loss += train_loss.item() * batch_size
        scaler.scale(train_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        del batch, outputs
    running_train_loss = torch.tensor([running_train_loss], device=device)
    train_epoch_loss = running_train_loss.item() / len(train_queue) #customdataset
    print(f"Epoch: {i}, Train Loss: {train_epoch_loss}")

Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-tiny-cityscapes-semantic and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([14]) in the model instantiated
- class_predictor.weight: found shape torch.Size([20, 256]) in the checkpoint and torch.Size([14, 256]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([20]) in the checkpoint and torch.Size([14]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO]: Extracting Railsem19 dataset from hub on
[INFO]: Total images: 8500
[INFO]: Total Training images: 8160
[INFO]: Total Validation images: 320
[INFO]: Total Test images: 20
[INFO]: Found labels_info.json.Skipping Download...
------------Epoch: 0-----------------
[WARN]: Patch List is empty...
[WARN]:Exception while getting image for patching: 'NoneType' object is not an iterator
dict_keys(['pixel_values', 'pixel_mask', 'mask_labels', 'class_labels', 'original_images', 'original_segmentation_maps'])
pixel_values: torch.Size([2, 3, 1024, 1024])
mask_labels: torch.Size([10, 1024, 1024]), torch.Size([9, 1024, 1024])
class_labels: tensor([ 0,  1,  2,  3,  4,  5,  7, 10, 11, 12]), tensor([ 0,  1,  2,  3,  4,  5, 10, 11, 12])
Step: 0, Train Loss: 113.28421020507812
dict_keys(['pixel_values', 'pixel_mask', 'mask_labels', 'class_labels', 'original_images', 'original_segmentation_maps'])
pixel_values: torch.Size([2, 3, 1024, 1024])
mask_labels: torch.Size([11, 1024, 1024]), torch.Size([12,

In [ ]:
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation,  AutoConfig
import torch.nn.functional as F
from torch import nn
import gc
from torch.cuda.amp import GradScaler, autocast


# id2label
id2label, label2id, labels = get_modified_labels()
# model and processor
model_ckpt = "facebook/mask2former-swin-tiny-cityscapes-semantic"
model_config = AutoConfig.from_pretrained(model_ckpt, id2label=id2label, label2id=label2id, num_labels=len(id2label))
model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)

processor = AutoImageProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False, do_resize=False) #384, 384 without do_resize=False
#default - "do_normalize": true, "do_rescale": true, "do_resize": true, "ignore_index": 255, "reduce_labels": false,
processor.num_labels = len(id2label)

#dataset
mydataset = load_railsem_dataset()
total_size = len(mydataset["data"])
print(f"[INFO]: Total images: {total_size}")
train_size = 8160
train_percentage = train_size / total_size
# Use train_test_split with specified percentages
splits = mydataset["data"].train_test_split(train_size=train_percentage, shuffle=True)
train_split = splits["train"]
val_split = splits["test"].select(indices=(range(320))) #320
test_split = splits["test"].select(indices=(range(20)))
print(f"[INFO]: Total Training images: {len(train_split)}")
print(f"[INFO]: Total Validation images: {len(val_split)}")
print(f"[INFO]: Total Test images: {len(test_split)}")

# transforms
transforms = get_transforms()
# customdataset = CustomDataset(train_split.select(range(0, 6)), transforms) #1080, 1920
customdataset = CustomDataset(train_split, transforms)

patch_size = [512, 512] # [512, 512] #[1024, 1024]
train_queue = PatchQueue(customdataset, max_length=10, samples_per_image=5, patch_size=patch_size) #1024, 1024
dataloader = _prepare_dataloader(train_queue) #train_queue #customdataset

device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.empty_cache()
    gc.collect()

optimizer = AdamW(params=model.parameters(), lr=1e-4)
scaler = GradScaler()
epochs = 1
print(f"Train Queue: {len(train_queue)}")

model.to(device)
model.train()
for i in range(epochs):
    print(f"------------Epoch: {i}-----------------")
    running_train_loss = 0.0
    for step, batch in enumerate(dataloader):
        print(batch.keys())
        print(f'pixel_values: {batch["pixel_values"].shape}')
        batch_dict = {
                "pixel_values": batch["pixel_values"].to(device),
                "mask_labels": [labels.to(device)
                                for labels in batch["mask_labels"]],
                "class_labels": [labels.to(device)
                                    for labels in batch["class_labels"]],
                "pixel_mask": batch["pixel_mask"].to(device),}
        # with torch.autocast(device_type='cpu', dtype=torch.bfloat16):
        with autocast():
            print(f'mask_labels: {batch["mask_labels"][0].shape}, {batch["mask_labels"][1].shape}') # mask_labels: torch.Size([8, 384, 384]), torch.Size([11, 384, 384])
            print(f'class_labels: {batch["class_labels"][0]}, {batch["class_labels"][1]}') # class_labels: tensor([ 2,  3,  4,  5,  7,  9, 10, 11]), tensor([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 11, 12])
            outputs = model(**batch_dict)
            train_loss = outputs.loss
        # print(outputs.keys())  # dict_keys(['loss', 'class_queries_logits', 'masks_queries_logits', 'encoder_last_hidden_state', 'pixel_decoder_last_hidden_state', 'transformer_decoder_last_hidden_state'])
        print(f"Step: {step}, Train Loss: {train_loss.item()}")
        batch_size = batch["pixel_values"].size(0)
        running_train_loss += train_loss.item() * batch_size
        scaler.scale(train_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        del batch, outputs
    running_train_loss = torch.tensor([running_train_loss], device=device)
    train_epoch_loss = running_train_loss.item() / len(train_queue) #customdataset
    print(f"Epoch: {i}, Train Loss: {train_epoch_loss}")

Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-tiny-cityscapes-semantic and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([14]) in the model instantiated
- class_predictor.weight: found shape torch.Size([20, 256]) in the checkpoint and torch.Size([14, 256]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([20]) in the checkpoint and torch.Size([14]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
dict_keys(['pixel_values', 'pixel_mask', 'mask_labels', 'class_labels', 'original_images', 'original_segmentation_maps'])
pixel_values: torch.Size([4, 3, 512, 512])
mask_labels: torch.Size([4, 512, 512]), torch.Size([6, 512, 512])
class_labels: tensor([ 2,  5, 11, 12]), tensor([ 2,  3,  4,  5, 11, 12])
Step: 8454, Train Loss: 97.7938461303711
[WARN]: Patch List is empty...
dict_keys(['pixel_values', 'pixel_mask', 'mask_labels', 'class_labels', 'original_images', 'original_segmentation_maps'])
pixel_values: torch.Size([4, 3, 512, 512])
mask_labels: torch.Size([10, 512, 512]), torch.Size([10, 512, 512])
class_labels: tensor([ 0,  2,  3,  4,  5,  7,  8, 10, 11, 12]), tensor([ 0,  1,  2,  3,  4,  5,  7, 10, 11, 12])
Step: 8455, Train Loss: 99.00200653076172
dict_keys(['pixel_values', 'pixel_mask', 'mask_labels', 'class_labels', 'original_images', 'original_segmentation_maps'])
pixel_values: torch.Size([4, 3, 512, 512])
mask_labels: torch.S

In [ ]:
print(guyyyhjkuijklh)
ik

In [ ]:
from transformers import AutoImageProcessor, AutoModelForUniversalSegmentation,  AutoConfig
import torch.nn.functional as F
from torch import nn
import gc
from torch.cuda.amp import GradScaler, autocast


# id2label
id2label, label2id, labels = get_modified_labels()
# model and processor
model_ckpt = "facebook/mask2former-swin-tiny-cityscapes-semantic"
model_config = AutoConfig.from_pretrained(model_ckpt, id2label=id2label, label2id=label2id, num_labels=len(id2label))
model = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt, ignore_mismatched_sizes=True, config=model_config)

# # Freeze the model backbone params
# for name, params in model.named_parameters():
#     if name.startswith("model.pixel_level_module"):
#         params.requires_grad = False

processor = AutoImageProcessor.from_pretrained(model_ckpt, ignore_index=255, do_reduce_labels=False)
#default - "do_normalize": true, "do_rescale": true, "do_resize": true, "ignore_index": 255, "reduce_labels": false,
processor.num_labels = len(id2label)

#dataset
mydataset = load_railsem_dataset()
total_size = len(mydataset["data"])
print(f"[INFO]: Total images: {total_size}")
train_size = 8160
train_percentage = train_size / total_size
# Use train_test_split with specified percentages
splits = mydataset["data"].train_test_split(train_size=train_percentage, shuffle=True)
train_split = splits["train"]
val_split = splits["test"].select(indices=(range(320))) #320
test_split = splits["test"].select(indices=(range(20)))
print(f"[INFO]: Total Training images: {len(train_split)}")
print(f"[INFO]: Total Validation images: {len(val_split)}")
print(f"[INFO]: Total Test images: {len(test_split)}")

# transforms
transforms = get_transforms()
# customdataset = CustomDataset(train_split.select(range(0, 6)), transforms)
customdataset = CustomDataset(train_split, transforms)

# patch_size = [512, 512]
# train_queue = PatchQueue(customdataset, max_length=10, samples_per_image=5, patch_size=patch_size)
dataloader = _prepare_dataloader(customdataset) #train_queue

# print(customdataset[0].keys(), customdataset[0]["image"].shape, customdataset[0]["mask"].shape)
# batch = next(iter(dataloader))
# for k,v in batch.items():
#   if isinstance(v, torch.Tensor):
#     print(k,v.shape)
#   else:
#     print(k,v[0].shape)

device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.empty_cache()
    gc.collect()

optimizer = AdamW(params=model.parameters(), lr=1e-4)
scaler = GradScaler()
epochs = 2

model.to(device)
model.train()
for i in range(epochs):
    print(f"------------Epoch: {i}-----------------")
    running_train_loss = 0.0
    for step, batch in enumerate(dataloader):
        print(batch.keys())
        batch_dict = {
                "pixel_values": batch["pixel_values"].to(device),
                "mask_labels": [labels.to(device)
                                for labels in batch["mask_labels"]],
                "class_labels": [labels.to(device)
                                    for labels in batch["class_labels"]],
                "pixel_mask": batch["pixel_mask"].to(device),}
        # with torch.autocast(device_type='cuda', dtype=torch.float16):
        with autocast():
            # print(f'mask_labels: {batch["mask_labels"][0].shape}, {batch["mask_labels"][1].shape}') # mask_labels: torch.Size([8, 384, 384]), torch.Size([11, 384, 384])
            # print(f'class_labels: {batch["class_labels"][0]}, {batch["class_labels"][1]}') # class_labels: tensor([ 2,  3,  4,  5,  7,  9, 10, 11]), tensor([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 11, 12])
            outputs = model(**batch_dict)
            train_loss = outputs.loss
        # print(outputs.keys()) # odict_keys(['loss', 'class_queries_logits', 'masks_queries_logits', 'encoder_last_hidden_state', 'pixel_decoder_last_hidden_state', 'transformer_decoder_last_hidden_state'])
        print(f"Step: {step}, Train Loss: {train_loss.item()}")
        batch_size = batch["pixel_values"].size(0)
        running_train_loss += train_loss.item() * batch_size

        scaler.scale(train_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        del batch, outputs
    running_train_loss = torch.tensor([running_train_loss], device=device)
    train_epoch_loss = running_train_loss.item() / len(customdataset)
    print(f"Epoch: {i}, Train Loss: {train_epoch_loss}")
